# XGBoost  + 'features'

## Train set

In [26]:
import pandas as pd
import numpy as np
from sklearn.metrics import log_loss
from sklearn.metrics import make_scorer
from sklearn.ensemble import GradientBoostingClassifier
# from xgboost import XGBClassifier
import xgboost as xgb
import cPickle
import warnings
warnings.filterwarnings("ignore")
from sklearn.grid_search import GridSearchCV
from sklearn.grid_search import RandomizedSearchCV
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import KFold
from sklearn.preprocessing import LabelEncoder
from time import time

pd.options.display.max_columns = 160

In [27]:
all_df = pd.read_json('train_feats_w_ids.json')
# all_df = pd.read_csv('train_feats_max_desc.csv')

In [28]:
all_df.shape
# (49352, 83)

(49352, 90)

In [29]:
le1 = LabelEncoder()
le1.fit(all_df['manager_id'])
all_df['manager_id'] = le1.transform(all_df['manager_id']) 

le2 = LabelEncoder()
le2.fit(all_df['building_id'])
all_df['building_id'] = le2.transform(all_df['building_id']) 

le3 = LabelEncoder()
le3.fit(all_df['display_address'])
all_df['display_address'] = le3.transform(all_df['display_address']) 

In [30]:
all_df.head(2)

,actual apt. photos,all utilities included,attended lobby,balcony,bathrooms,bedrooms,bike room,brownstone,building_id,business center,cable/satellite tv,central a/c,childrens playroom,community recreation facilities,created,description,dining room,dishwasher,display_address,doorman,duplex,elevator,exclusive,exposed brick,features,fireplace,fitness center,flex-2,furnished,garage,garden/patio,granite kitchen,green building,guarantors accepted,hardwood floors,high ceilings,high speed internet,highrise,index,interest_level,latitude,laundry in building,light,listing_id,live in super,live/work,loft,longitude,lowrise,manager_id,marble bath,microwave,midrise,multi-level,new construction,no fee,no pets,num_desc,num_feats,num_photos,outdoor space,package room,parking space,pet friendly,pets on approval,post-war,pre-war,price,private outdoor space,reduced fee,renovated,residents lounge,roof deck,sauna,shares ok,short term allowed,simplex,skylight,stainless steel appliances,storage,sublet,subway,swimming pool,terrace,valet,video intercom,view,virtual doorman,walk in closet(s),wheelchair access
0,0,0,0,0,1.5,3,0,0,2431,0,0,0,0,0,1466754864,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,0,0,6544,0,0,0,0,0,[],0,0,0,0,0,0,0,0,0,0,0,0,0,10,medium,40.7145,0,0,7211212,0,0,0,-73.9425,0,1239,0,0,0,0,0,0,0,588,0,5,0,0,0,0,0,0,0,3000,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,1.0,2,0,0,5862,0,0,0,0,0,1465733967,,0,0,4506,1,0,1,0,0,"[Doorman, Elevator, Fitness Center, Cats Allow...",0,1,0,0,0,0,0,0,0,0,0,0,0,10000,low,40.7947,0,0,7150865,0,0,0,-73.9667,0,1583,0,0,0,0,0,0,0,8,5,11,0,0,0,1,0,0,0,5465,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [81]:
x_train, x_val, y_train, y_val = train_test_split(all_df.drop(['interest_level'], 1),all_df[['interest_level']], test_size=0.2, random_state=42)
# x_train = all_df.drop(['interest_level'], 1)
# y_train = all_df[['interest_level']]

In [82]:
cat_feats = cPickle.load(open('cat_feats.p', 'rb'))

for col in ['interest_level']:
    y_train[col] = y_train[col].astype('category')
    y_val[col] = y_val[col].astype('category')
    
for col in cat_feats+['manager_id', 'building_id']:
    x_train[col] = x_train[col].astype('category')
    x_val[col] = x_val[col].astype('category')

In [83]:
drop_list = [u'listing_id', 'index', 'created', 'features', 'description']
x_train_small = x_train.drop(drop_list,1)

x_val_small = x_val.drop(drop_list,1)

In [84]:
# low_pvalues_cols = cPickle.load(open('low_pvalues_cols.p', 'rb'))
x_train_best = x_train_small#[low_pvalues_cols]
x_val_best = x_val_small#[low_pvalues_cols]
x_train_best.shape
# x_train_best.head(2)

(39481, 84)

In [85]:
# for col in x_train_best.columns:
#     x_train_best[col] = x_train_best[col].astype('float')
#     x_val_best[col] = x_val_best[col].astype('float')

# le5 = LabelEncoder()
# le5.fit(y_train['interest_level'])
# y_train['interest_level'] = le5.transform(y_train['interest_level'])

# le5.fit(y_val['interest_level'])
# y_val['interest_level'] = le5.transform(y_val['interest_level']) 

# for col in y_train.columns:
#     y_train[col] = y_train[col].astype('float')
#     y_val[col] = y_val[col].astype('float')
target_num_map = {'high':0, 'medium':1, 'low':2}
y_train = y_train['interest_level'].apply(lambda x: target_num_map[x])
y_val = y_val['interest_level'].apply(lambda x: target_num_map[x])

In [61]:
# y_val[y_val == 2] = 0
# y_train[y_train == 2] = 0

In [86]:
xgtrain = xgb.DMatrix(x_train_best.values, label=y_train.values)
xgval = xgb.DMatrix(x_val_best.values, label=y_val.values)

In [95]:
param = {'bst:max_depth':2, 'bst:eta':1, 'silent':1, 'objective':'multi:softprob', 'num_class':3}
param['nthread'] = 3
plst = param.items()
plst += [('eval_metric', 'mlogloss')] # Multiple evals can be handled in this way

In [96]:
evallist  = [(xgval,'eval'), (xgtrain,'train')]

In [97]:
np.unique(xgtrain.get_label())

array([ 0.,  1.,  2.], dtype=float32)

In [99]:
num_round = 100
model = xgb.train( plst, xgtrain, num_round, evallist, early_stopping_rounds=20 )

Will train until train error hasn't decreased in 20 rounds.
[0]	eval-mlogloss:0.766817	train-mlogloss:0.763243
[1]	eval-mlogloss:0.714359	train-mlogloss:0.708135
[2]	eval-mlogloss:0.686335	train-mlogloss:0.680187
[3]	eval-mlogloss:0.672804	train-mlogloss:0.667008
[4]	eval-mlogloss:0.660741	train-mlogloss:0.655715
[5]	eval-mlogloss:0.654661	train-mlogloss:0.649446
[6]	eval-mlogloss:0.647251	train-mlogloss:0.642099
[7]	eval-mlogloss:0.642342	train-mlogloss:0.637851
[8]	eval-mlogloss:0.635747	train-mlogloss:0.631747
[9]	eval-mlogloss:0.632150	train-mlogloss:0.627733
[10]	eval-mlogloss:0.628672	train-mlogloss:0.624152
[11]	eval-mlogloss:0.626279	train-mlogloss:0.621246
[12]	eval-mlogloss:0.624056	train-mlogloss:0.618061
[13]	eval-mlogloss:0.623110	train-mlogloss:0.616278
[14]	eval-mlogloss:0.621818	train-mlogloss:0.614692
[15]	eval-mlogloss:0.620441	train-mlogloss:0.612633
[16]	eval-mlogloss:0.618294	train-mlogloss:0.610574
[17]	eval-mlogloss:0.617053	train-mlogloss:0.609006
[18]	eval-mlog

In [70]:
# model = GradientBoostingClassifier(n_estimators=500, learning_rate=0.1, max_depth=4, max_features=0.3, random_state=0)
# # n_estimator: 500; learning_rate: 0.2; max_features: 0.4; max_depth: 3
# model = model.fit(x_train_best, y_train)
# # model = model.fit(x_train_best[x_train_best['manager_id'] != '9df32cb8dda19d3222d66e69e258616b'], y_train[x_train_best['manager_id'] != '9df32cb8dda19d3222d66e69e258616b'])

In [102]:
predicted_train = pd.DataFrame(model.predict(xgtrain))
# predicted = model.predict_proba(x)
predicted_train.columns = ['high', 'low', 'medium']
predicted_train.head()

,high,low,medium
0,0.196853,0.281979,0.521168
1,0.169821,0.438557,0.391622
2,0.009123,0.070467,0.920410
3,0.003521,0.007375,0.989104
4,0.888784,0.050124,0.061092


In [103]:
log_loss_train = log_loss(y_train, predicted_train.as_matrix())
log_loss_train

0.54028868104338434

In [105]:
predicted_val = pd.DataFrame(model.predict(xgval))
# predicted = model.predict_proba(x)
predicted_val.columns = ['high', 'low', 'medium']
predicted_val.head()
# predicted

,high,low,medium
0,0.011596,0.106089,0.882315
1,0.132497,0.192935,0.674568
2,0.006193,0.050331,0.943476
3,0.145186,0.385074,0.469741
4,0.000332,0.015096,0.984572


In [106]:
log_loss_val = log_loss(y_val, predicted_val.as_matrix())
log_loss_val

0.57619896134104442

In [45]:
a = zip(x_train_best.columns, list(model.feature_importances_))
a.sort(key = lambda t: t[1], reverse=True)
a 

AttributeError: 'XGBClassifier' object has no attribute 'feature_importances_'

In [88]:
# bad_feats_gb = [i[0] for i in a[-20:]]
# cPickle.dump(bad_feats_gb, open('bad_feats_gb.p', 'wb')) 
bad_feats_gb = cPickle.load(open('bad_feats_gb.p', 'rb'))
len(bad_feats_gb)

20

In [89]:
x_train_best.drop(bad_feats_gb, axis=1, inplace=True)
x_val_best.drop(bad_feats_gb, axis=1, inplace=True)

In [90]:
x_train_best.shape, x_val_best.shape

((39481, 64), (9871, 64))

In [91]:
a

[(u'price', 0.1403876354221949),
 (u'manager_id', 0.1029365337795453),
 (u'latitude', 0.10046600345742292),
 (u'longitude', 0.091100706092862707),
 (u'display_address', 0.080850508061234658),
 (u'building_id', 0.071903651343100491),
 (u'num_desc', 0.064868056535091506),
 (u'bedrooms', 0.04635575136096131),
 (u'num_feats', 0.036215129188849483),
 (u'num_photos', 0.030450993487031071),
 (u'bathrooms', 0.016492983218515056),
 (u'no fee', 0.012490006374658925),
 (u'reduced fee', 0.0073349764569794959),
 (u'pet friendly', 0.0066465987051817256),
 (u'hardwood floors', 0.0060925020627741588),
 (u'pre-war', 0.0059066467686295521),
 (u'laundry in building', 0.0057772566784935909),
 (u'private outdoor space', 0.0057527249992516448),
 (u'view', 0.0056238566756772477),
 (u'elevator', 0.0048210584567277941),
 (u'furnished', 0.004758354433627641),
 (u'outdoor space', 0.0047068301140478904),
 (u'dining room', 0.00462086354606409),
 (u'light', 0.004564215830092859),
 (u'fitness center', 0.004261716568

## Randomized Search

In [13]:
log_scoring=make_scorer(log_loss, greater_is_better=False, needs_proba=True)
log_scoring

make_scorer(log_loss, greater_is_better=False, needs_proba=True)

In [23]:
clf = GradientBoostingClassifier()
# specify parameters and distributions to sample from
param_dist = {"max_depth": [1, 2, 3, 4, 5, 6, 7, 8],
              "max_features": ['sqrt',0.2, 0.3, 0.4, 0.5],
              "n_estimators": [300, 400, 500, 600, 700, 800],
              "learning_rate": [round(i,2) for i in np.arange(0.05,0.2,0.01)]
             "min_samples_split": [None, 100, 300, 400, 500, 600, 800, 1000]
             "min_samples_leaf": [None, 20, 40, 50, 70 ,100]}

# run randomized search
n_iter_search = 10
random_search = RandomizedSearchCV(clf, param_distributions=param_dist,
                                   n_iter=n_iter_search, cv=5, n_jobs=3, scoring=log_scoring, verbose=1)

In [24]:
start = time()
search = random_search.fit(x_train_best, y_train['interest_level'])
print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time() - start), n_iter_search))
# report(random_search.cv_results_)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  4.6min
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed: 28.9min
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed: 36.8min finished


RandomizedSearchCV took 2278.45 seconds for 50 candidates parameter settings.


In [107]:
# search.grid_scores_

In [26]:
search.best_score_, search.best_estimator_, search.best_params_

(-0.5933256136564423,
 GradientBoostingClassifier(criterion='friedman_mse', init=None,
               learning_rate=0.1, loss='deviance', max_depth=4,
               max_features=0.3, max_leaf_nodes=None,
               min_impurity_split=1e-07, min_samples_leaf=1,
               min_samples_split=2, min_weight_fraction_leaf=0.0,
               n_estimators=500, presort='auto', random_state=None,
               subsample=1.0, verbose=0, warm_start=False),
 {'learning_rate': 0.1,
  'max_depth': 4,
  'max_features': 0.3,
  'n_estimators': 500})

In [149]:
x_train_best.shape, y_train.shape

((39481, 60), (39481, 1))

## Cross Validation

In [152]:
def get_cv_loss(ne, lr, md, mf):
    kf = KFold(x_train_best.shape[0], n_folds=5, random_state=2017)
    loss_tr = []
    loss_ts = []
    for train_index, test_index in kf:
        x_tr = x_train_best.reset_index().loc[train_index].set_index(['index'])
        y_tr =  y_train.reset_index().loc[train_index].set_index(['index'])
        x_ts = x_train_best.reset_index().loc[test_index].set_index(['index'])
        y_ts = y_train.reset_index().loc[test_index].set_index(['index'])

        model = GradientBoostingClassifier(n_estimators=ne, learning_rate=lr, max_depth=md, max_features=mf, random_state=20)

        model = model.fit(x_tr, y_tr)

        predicted_ts = pd.DataFrame(model.predict_proba(x_ts))
        predicted_ts.columns = ['high', 'low', 'medium']
        log_loss_ts = log_loss(y_ts, predicted_ts.as_matrix())
        loss_ts.append(log_loss_ts)
    
    print 'CV loss:', np.mean(loss_ts)
    predicted_val = pd.DataFrame(model.predict_proba(x_val_best))
    predicted_val.columns = ['high', 'low', 'medium']
    log_loss_val = log_loss(y_val, predicted_val.as_matrix())
    print 'Val loss:', log_loss_val
    return np.mean(loss_ts)

In [153]:
n_estimators = [500]
learning_rates = [0.4, 0.6, 0.8, 1]
max_features = ['sqrt',0.2, 0.4, 0.8]
max_depth = [1, 3, 5]#, 12, 14, 16, 18, 20]
best_loss = 100
best_params = ''
count = 50
for ne in n_estimators:
    for lr in learning_rates:
        for md in max_depth:
            for mf in max_features:
                print count
                count += 1
                params = 'n_estimator: ' + str(ne) + '; learning_rate: ' + str(lr) + '; max_features: ' + str(mf) + '; max_depth: ' + str(md)
                print params
    #             model = RandomForestClassifier(n_estimators=500, max_features=mf, max_depth=md, min_samples_split=mss, random_state=0)
    #             model = model.fit(x_train_best, y_train)
    #             predicted_train = pd.DataFrame(model.predict_proba(x_train_best))
    #             predicted_train.columns = ['high', 'low', 'medium']
    #             log_loss_train = log_loss(y_train, predicted_train.as_matrix())
    #             print 'Train loss:', log_loss_train
    #             predicted_val = pd.DataFrame(model.predict_proba(x_val_best))
    #             predicted_val.columns = ['high', 'low', 'medium']
    #             log_loss_val = log_loss(y_val, predicted_val.as_matrix())
    #             print 'Val loss:', log_loss_val
    #             if log_loss_val < best_loss:
    #                 best_loss = log_loss_val
    #                 best_params = params
                cv_loss = get_cv_loss(ne, lr, md, mf)
                if cv_loss < best_loss:
                    best_loss = cv_loss
                    best_params = params   

print 'best_loss:', best_loss
print 'best_params:', best_params
# best_loss: 0.638016960424
# best_params: max_features: 0.3; max_depth: 18; min_samples_split: 20

50
n_estimator: 500; learning_rate: 0.4; max_features: sqrt; max_depth: 1
CV loss: 0.625799776246
Val loss: 0.628988463414
51
n_estimator: 500; learning_rate: 0.4; max_features: 0.2; max_depth: 1
CV loss: 0.624416750049
Val loss: 0.622868663987
52
n_estimator: 500; learning_rate: 0.4; max_features: 0.4; max_depth: 1
CV loss: 0.624925251005
Val loss: 0.620767325559
53
n_estimator: 500; learning_rate: 0.4; max_features: 0.8; max_depth: 1
CV loss: 0.626268963256
Val loss: 0.620891598599
54
n_estimator: 500; learning_rate: 0.4; max_features: sqrt; max_depth: 3


KeyboardInterrupt: 



## Competition Test Set

In [108]:
test_df = pd.read_json('test_feats_w_ids.json')

In [109]:
le4 = LabelEncoder()
le4.fit(test_df['manager_id'])
test_df['manager_id'] = le4.transform(test_df['manager_id']) 

le5 = LabelEncoder()
le5.fit(test_df['building_id'])
test_df['building_id'] = le5.transform(test_df['building_id']) 

le6 = LabelEncoder()
le6.fit(test_df['display_address'])
test_df['display_address'] = le6.transform(test_df['display_address']) 

In [110]:
cat_feats = cPickle.load(open('cat_feats.p', 'rb'))

for col in cat_feats+['manager_id', 'building_id']:
    test_df[col] = test_df[col].astype('category')

In [111]:
x_test = test_df[x_train_best.columns]
x_test.shape

(74659, 84)

In [112]:
x_test.head(2)

,actual apt. photos,all utilities included,attended lobby,balcony,bathrooms,bedrooms,bike room,brownstone,building_id,business center,cable/satellite tv,central a/c,childrens playroom,community recreation facilities,dining room,dishwasher,display_address,doorman,duplex,elevator,exclusive,exposed brick,fireplace,fitness center,flex-2,furnished,garage,garden/patio,granite kitchen,green building,guarantors accepted,hardwood floors,high ceilings,high speed internet,highrise,latitude,laundry in building,light,live in super,live/work,loft,longitude,lowrise,manager_id,marble bath,microwave,midrise,multi-level,new construction,no fee,no pets,num_desc,num_feats,num_photos,outdoor space,package room,parking space,pet friendly,pets on approval,post-war,pre-war,price,private outdoor space,reduced fee,renovated,residents lounge,roof deck,sauna,shares ok,short term allowed,simplex,skylight,stainless steel appliances,storage,sublet,subway,swimming pool,terrace,valet,video intercom,view,virtual doorman,walk in closet(s),wheelchair access
0,0,0,0,0,1.0,1,0,0,4412,0,0,0,0,0,0,1,9506,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,40.7185,1,0,0,0,0,-73.9865,0,2694,0,0,0,0,0,0,0,587,6,8,1,0,0,0,0,0,0,2950,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0
1,0,0,0,0,1.0,2,0,0,0,0,0,0,0,0,0,0,9589,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,40.7278,0,0,0,0,0,-74.0000,0,3145,1,0,0,0,0,0,0,245,3,3,0,0,0,1,0,0,1,2850,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0


In [113]:
xgtest = xgb.DMatrix(x_test.values)#, label=y_val.values)

In [119]:
pred_x = pd.DataFrame(model.predict(xgtest))
# pred_x.columns = ['high', 'low', 'medium']
pred_x.head()

,0,1,2
0,0.133795,0.510345,0.355860
1,0.116086,0.234070,0.649843
2,0.002688,0.036486,0.960826
3,0.281814,0.536613,0.181574
4,0.012394,0.029491,0.958115


In [115]:
subm = pd.merge(test_df[['listing_id']].reset_index(), pred_x.reset_index(), left_index=True, right_index=True)

In [116]:
subm = subm[['listing_id', 'high', 'medium', 'low']]

In [117]:
subm.shape
# (74659, 4)

(74659, 4)

In [118]:
subm.to_csv('Submission_myXGB1.csv', index=None)

In [86]:
model

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=4,
              max_features=0.3, max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=1,
              min_samples_split=2, min_weight_fraction_leaf=0.0,
              n_estimators=500, presort='auto', random_state=0,
              subsample=1.0, verbose=0, warm_start=False)